<a href="https://colab.research.google.com/github/yas1124/howdy.github.io/blob/main/flux.1-dev_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Google Driveをマウント
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

save_dir = "/content/drive/MyDrive/FLUX_outputs"
os.makedirs(save_dir, exist_ok=True)

# 2. 本体（TotoroUI）のセットアップ
%cd /content
!rm -rf /content/TotoroUI
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

# 3. 依存ライブラリのクリーンインストール
# 一旦問題を起こしている torchvision を消して、適切なバージョンを入れ直します
!pip uninstall -y torchvision torchaudio
!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2
!pip install -q torchvision==0.20.0 --no-deps
!apt -y install -qq aria2

# 4. モデルのダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/TotoroUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/TotoroUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors

# 5. インポートと初期化
import sys
sys.path.append('/content/TotoroUI')
import torch
import numpy as np
import random
from PIL import Image
from datetime import datetime

import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

print("--- 準備完了！ステップ2を実行してください ---")

Mounted at /content/drive
/content
Cloning into '/content/TotoroUI'...
remote: Enumerating objects: 20484, done.
remote: Total 20484 (delta 0), reused 0 (delta 0), pack-reused 20484 (from 1)
Receiving objects: 100% (20484/20484), 65.43 MiB | 26.60 MiB/s, done.
Resolving deltas: 100% (13661/13661), done.
/content/TotoroUI
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 906.4/906.4 MB 194.5 MB/s eta 0:00:01

In [1]:
with torch.inference_mode():
    # --- 設定項目 ---
    # 添付画像を元にしたプロンプト
    positive_prompt = """
    Cinematic, hyper-realistic photo of a beautiful young Japanese woman standing waist-deep in a crystal-clear tropical lagoon,
    reaching out one hand toward the viewer in a warm, welcoming gesture. She is wearing an ethereal fantasy dress made of
    flowing light pink fabric, intricately adorned with iridescent seashells, glowing pearls, and delicate coral textures.
    Beneath the shimmering water surface, vibrant coral reefs and colorful tropical fish are visible.
    The background features a serene white-sand beach with swaying palm trees under a bright, golden afternoon sun.
    High detail, 8k, emotional and hopeful atmosphere, soft natural bokeh, masterpiece.
    """

    width = 1024
    height = 1024
    seed = 0         # 0でランダム生成
    steps = 30       # ご指定のステップ数
    guidance = 3.5   # ご指定のCFGスケール (Flux Guidance)

    sampler_name = "euler"
    scheduler = "simple"
    # ---------------

    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"生成開始 (Steps: {steps}, Guidance: {guidance}, Seed: {seed})")

    # 1. プロンプトのエンコード
    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)

    # Guidance（CFGスケール）の設定を条件データに追加
    cond = [[cond, {"pooled_output": pooled, "guidance": guidance}]]

    # 2. 生成プロセスの実行
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]

    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)

    model_management.soft_empty_cache()

    # 3. 画像の復元と保存
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    final_image = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])

    # ファイル名作成（日時+シード）
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"flux_fantasy_{timestamp}_{seed}.png"
    save_path = os.path.join(save_dir, filename)

    # Google Driveに保存
    final_image.save(save_path)
    print(f"完了！Google Driveに保存されました: {save_path}")

# 表示
final_image

NameError: name 'torch' is not defined